<a href="https://colab.research.google.com/github/Sinha123456/Work-Projects/blob/main/Discipline_SigDispro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

#### LEA from Sig dispor calculator

In [ ]:
master_corp = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/corp_no_from_sig_dispro_calc.xlsx')

In [ ]:
all_corps = master_corp['Reporting Corp'].unique()

In [ ]:
df1 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/ES/DOE - Sinha 2023-24 ES 11142024.xlsx")
#df1.head(1)

In [ ]:
df1.shape

In [ ]:
#df1.info()

In [ ]:
df1.columns = df1.columns.str.lower()
#df1.head(1)

In [ ]:
df1.nunique()

In [ ]:
df1.sort_values(by = "stn", ascending = False)
#df1.head(5)

In [ ]:
duplicated_mask = df1.duplicated(keep = False)


In [ ]:
#duplicated_mask # it's bollean value

In [ ]:
duplicated_mask.sum()

In [ ]:
df1_sorted = pd.concat([df1[duplicated_mask], df1[~duplicated_mask]])


In [ ]:
#df1_sorted.head(20)

In [ ]:
#df1.query("stn == 'N00008313'")

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
df1_sorted.shape

In [ ]:
#df1_sorted.head(5)

In [ ]:
#df1.query("stn == '028913079'")

In [ ]:
df2 = df1.drop_duplicates(subset = ["stn", "incident_date"], keep = "first")
df2.shape

In [ ]:
#df2.query("stn == '004110110'")

In [ ]:
#calculate age from birthdate column
from datetime import date
df2['Age'] = (date.today().year -pd.to_datetime(df2['birth_date']).dt.year)

In [ ]:
df3 = df2.query("Age >=3 and Age <= 21")

In [ ]:
df3.shape

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
#check columns discipline action length and actual discipline length columns
comparison = df3['discipline_action_length'] ==df3['actual_discipline_action_length']

In [ ]:
print(comparison)

In [ ]:
#sum the discipline action length of each student
#df3 = df3.groupby('stn')['discipline_action_length'].sum()

In [ ]:
#df3.ethnicity.value_counts()

In [ ]:
df3.replace({'White':'WH7', 'Black':'BL7', 'Hispanic': 'HI7', 'Multiracial': 'MU7', 'Asian' : 'AS7', 'American Indian' : 'AM7', 'Native Hawaiian or Other Pacific Islander' : 'PI7'}, inplace = True)

In [ ]:
df3.ethnicity.value_counts()

In [ ]:
df3.special_ed.value_counts()

In [ ]:
#Only Special ed students
df3_sped = df3.query("special_ed =='Y'")

In [ ]:
# non special ed students
df3_Nonsped = df3.query("special_ed =='N'")

##Comparison Discipline Sheet

In [ ]:
#Comparison group:Total IDEA student enrollment-for Comparison Discipline sheet
target_idea_student = df3_sped.groupby('idoe_corporation_id')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = "idoe_corporation_id").reindex(all_corps, fill_value=0)

In [ ]:
target_idea_student

In [ ]:
#target_idea_student.to_csv("Comparison_discipline_2024.csv")

##In School Suspension

In [ ]:
#filter in school suspension
df3_sped_in_school_suspension = df3_sped.query("discipline_type =='In School Suspension'")

####Sum discipline action length - In school suspension

In [ ]:

df3_sped_in_school_suspension_sum = df3_sped_in_school_suspension.groupby('stn')['discipline_action_length'].sum()

####In school suspension less than or equal to 10 days

In [ ]:
df3_sped_in_school_suspension_less_than_10 = df3_sped_in_school_suspension_sum[df3_sped_in_school_suspension_sum <= 10].index

In [ ]:
df3_sped_in_school_suspension_less_than_10

#####Get corporation id and ethnicity of in school suspension for less than or equal to 10 days

In [ ]:
in_sch_less_than_10 = df3_sped_in_school_suspension[df3_sped_in_school_suspension['stn'].isin(df3_sped_in_school_suspension_less_than_10)][['idoe_corporation_id','ethnicity', 'stn']]

In [ ]:
#drop duplicate
in_sch_less_than_10 = in_sch_less_than_10.drop_duplicates(subset = ['stn'])

#####Group by corporation id, count ethnicity in school suspension less than or equal to 10 days

In [ ]:
final_in_sch_sus_less_than_10 = in_sch_less_than_10.groupby('idoe_corporation_id')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = "idoe_corporation_id").reindex(all_corps, fill_value=0)

In [ ]:
final_in_sch_sus_less_than_10

#####saved in school suspension less than or equal to 10 days, remove hash for future download

In [ ]:
#final_in_sch_sus_less_than_10.to_csv("In_school_suspension_less_than_10.csv")

##In School Suspension greater than or equal to 10 days

In [ ]:
in_sch_greater_than_10 = df3_sped_in_school_suspension_sum[df3_sped_in_school_suspension_sum >= 10].index

In [ ]:
in_sch_greater_than_10

In [ ]:
in_sch_greater_than_10_filter = df3_sped_in_school_suspension[df3_sped_in_school_suspension['stn'].isin(in_sch_greater_than_10)][['idoe_corporation_id','ethnicity', 'stn']]

In [ ]:
in_sch_greater_than_10_filter = in_sch_greater_than_10_filter.drop_duplicates(subset = ['stn'])

In [ ]:
final_in_sch_sus_more_than_10 = in_sch_greater_than_10_filter.groupby('idoe_corporation_id')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = "idoe_corporation_id").reindex(all_corps, fill_value=0)

####Remove hash for future download in school suspension greater than or equal to 10 days

In [ ]:
#final_in_sch_sus_more_than_10.to_csv("In_school_suspension_greater_than_10.csv")

#Out of School suspension/expulsion filter

In [ ]:
df3_sped_oss = df3_sped.query("discipline_type =='Out of School Suspension' or discipline_type == 'Expulsion'") # extract oss and expulsion

In [ ]:
df3_sped_oss_sum = df3_sped_oss.groupby('stn')['discipline_action_length'].sum() # sum the discipline action length

#####out of school suspension less than or equal to 10 days

In [ ]:
df3_sped_oss_less_than_10 = df3_sped_oss_sum[df3_sped_oss_sum <= 10].index

In [ ]:
df3_sped_oss_less_than_10

In [ ]:
#df3_sped.query("stn =='002516012'") # validate the stn

In [ ]:
df3_sped__oss_less_than_10_filter = df3_sped_oss[df3_sped_oss['stn'].isin(df3_sped_oss_less_than_10)][['idoe_corporation_id', 'ethnicity', 'stn']] # filter oss stn less than or equal to 10 days

In [ ]:
df3_sped__oss_less_than_10_filter = df3_sped__oss_less_than_10_filter.drop_duplicates(subset = ['stn'])

In [ ]:
df3_sped__oss_less_than_10_final = df3_sped__oss_less_than_10_filter.groupby('idoe_corporation_id')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = 'idoe_corporation_id').reindex(all_corps, fill_value=0)

####Save the file out of school suspension or expulsion less than or equal to 10 days. Remove the hash for download

In [ ]:
#df3_sped__oss_less_than_10_final.to_csv("df3_sped_oss_less_than_10_days.csv")

###Out of school suspension or Expulsion greater than or equal to 10 days

In [ ]:
df3_sped_oss_greater_than_10 = df3_sped_oss_sum[df3_sped_oss_sum >= 10].index

In [ ]:
df3_sped_oss_greater_than_10

In [ ]:
df3_sped_oss_greater_than_10_filter = df3_sped_oss[df3_sped_oss['stn'].isin(df3_sped_oss_greater_than_10)][['idoe_corporation_id', 'ethnicity', 'stn']]

In [ ]:
df3_sped_oss_greater_than_10_filter = df3_sped_oss_greater_than_10_filter.drop_duplicates(subset = ['stn'])

In [ ]:
df3_sped_oss_greater_than_10_final = df3_sped_oss_greater_than_10_filter.groupby('idoe_corporation_id')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = 'idoe_corporation_id').reindex(all_corps, fill_value=0)

####Save the file out of school suspension or expulsion greater than or equal to 10 days. Remove hash for download the file

In [ ]:
#df3_sped_oss_greater_than_10_final.to_csv("df3_sped_oss_greater_than_10.csv")

###Total Removal age 3-21 by race
count total student without summation of length of suspension

filter the arrest values.

In [ ]:
df3_sped.head(1)

In [ ]:
df3_sped_arrest_nan = df3_sped[df3_sped['arrest_reason'].isna()]

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
df3_sped_arrest_nan['arrest_reason'].value_counts()

In [ ]:
total_removal_idea_3_21 = df3_sped_arrest_nan.groupby('idoe_corporation_id')['ethnicity'].value_counts().unstack(fill_value=0).sort_values(by = "idoe_corporation_id").reindex(all_corps, fill_value=0)

### Save the file

In [ ]:
#total_removal_idea_3_21.to_excel("total_removal_idea_3_21.xlsx")
total_removal_idea_3_21

In [ ]:
in_sus_less_10 = in_sch_less_than_10.groupby(['idoe_corporation_id', 'stn'])['ethnicity'].value_counts().unstack(fill_value=0)

In [ ]:
in_sus_less_10.head()

In [ ]:
in_sch_less_than_10.query("stn == '251212258'") # Data verification

In [ ]:
#in_sus_less_10.to_csv("in_sus_less_10_Angie.csv")

In [ ]:
oos_greater_10_stn = df3_sped_oss_greater_than_10_filter.groupby(['idoe_corporation_id','stn'])['ethnicity'].value_counts().unstack(fill_value=0)

In [ ]:
df3_sped_oss_greater_than_10_filter.query("stn == '251212258'") # Data verification

In [ ]:
oos_greater_10_stn.to_csv("oos_greater_10_stn.csv")

In [ ]:
oos_greater_10_stn.query("stn == '004109502'") # data verification

In [ ]:
total_removal_idea_3_21_stn = df3_sped_arrest_nan.groupby(['stn', 'idoe_corporation_id'])['ethnicity'].value_counts().unstack(fill_value=0)

In [ ]:
total_removal_idea_3_21_stn

In [ ]:
total_removal_idea_3_21_stn.to_csv("total_removal_idea_3_21_stn.csv")